In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pyarrow.feather as feather
import seaborn as sns
import statsmodels.api as sm

from statsmodels.tsa.stattools import adfuller, coint
from scipy.optimize import minimize
from datetime import datetime, timedelta
from IPython.core.interactiveshell import InteractiveShell
from matplotlib.ticker import ScalarFormatter

InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
# price_df = feather.read_feather('data\\91_coin_price.feather')
# volume_df = feather.read_feather('data\\91_coin_volume.feather')
price_df = feather.read_feather('data/91_coin_price.feather')
volume_df = feather.read_feather('data/91_coin_volume.feather')

In [4]:
price_df

,KRW-BTC,KRW-1INCH,KRW-ADA,KRW-AERGO,KRW-AHT,KRW-ALGO,KRW-ANKR,KRW-APT,KRW-AQT,KRW-ARDR,...,KRW-TT,KRW-UPP,KRW-VET,KRW-WAVES,KRW-WAXP,KRW-XEM,KRW-XLM,KRW-XRP,KRW-XTZ,KRW-ZRX
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,21081000.0000,489.0000,314.0000,113.0000,5.4200,220.0000,19.6000,4390.0000,992.0000,84.3000,...,3.9500,58.7000,20.1000,1690.0000,53.1000,36.5000,90.6000,433.0000,915.0000,188.0000
2023-01-01 00:01:00,21061000.0000,490.0000,313.0000,113.0000,5.4200,220.0000,19.7000,4410.0000,994.0000,84.3000,...,3.9700,58.6000,20.2000,1670.0000,53.3000,36.5000,90.6000,432.0000,913.0000,188.0000
2023-01-01 00:02:00,21062000.0000,490.0000,314.0000,113.0000,5.4300,219.0000,19.6000,4410.0000,993.0000,84.4000,...,3.9500,58.4000,20.2000,1665.0000,53.3000,36.5000,90.6000,432.0000,915.0000,188.0000
2023-01-01 00:03:00,21064000.0000,490.0000,313.0000,113.0000,5.5000,219.0000,19.6000,4410.0000,989.0000,84.2000,...,3.9500,58.5000,20.2000,1665.0000,53.3000,36.5000,90.6000,432.0000,915.0000,189.0000
2023-01-01 00:04:00,21064000.0000,488.0000,313.0000,113.0000,5.4800,219.0000,19.6000,4405.0000,989.0000,84.1000,...,3.9500,58.5000,20.2000,1655.0000,53.3000,36.5000,90.5000,432.0000,915.0000,188.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-27 23:55:00,96141000.0000,588.1000,647.6000,173.4000,10.0400,272.6000,64.1400,13340.0000,1885.0000,144.7000,...,6.4000,113.9000,50.5600,3416.0000,95.2600,53.2500,152.8000,739.1000,1380.0000,773.3000
2024-05-27 23:56:00,96151000.0000,588.1000,647.6000,173.4000,10.0500,272.1000,64.0600,13340.0000,1885.0000,144.8000,...,6.4000,113.9000,50.5600,3422.0000,95.0500,53.2500,152.8000,739.0000,1380.0000,772.9000
2024-05-27 23:57:00,96153000.0000,589.5000,647.7000,173.4000,10.0500,272.6000,64.1500,13360.0000,1885.0000,144.7000,...,6.4000,113.9000,50.6800,3422.0000,95.1100,53.2600,152.8000,739.0000,1379.0000,774.0000


In [ ]:
n_cols = [i[4:] for i in price_df.columns]

In [ ]:
price_df.columns = n_cols
volume_df.columns = n_cols

In [ ]:
volume_df.sum().sort_values(ascending=False).head(10) / 513

In [ ]:
top6_volume_coins = ['BTC', 'XRP', 'SOL', 'DOGE', 'ETC', 'ETH']
agg_dict = {col:'last' for col in price_df.columns}

In [ ]:
price_df = price_df.resample('8H').agg(agg_dict)

In [ ]:
top6_df = price_df[top6_volume_coins]

btc_price = price_df['BTC']
xrp_price = price_df['XRP']
sol_price = price_df['SOL']
doge_price = price_df['DOGE']
etc_price = price_df['ETC']
eth_price = price_df['ETH']

top6_list = [btc_price, xrp_price, sol_price, doge_price, etc_price, eth_price]

In [ ]:
desc_stats_price = top6_df.describe()
desc_stats_price.loc['skewness'] = top6_df.skew()
desc_stats_price.loc['kurtosis'] = top6_df.kurtosis()

In [ ]:
# desc_stats_price.to_csv("desc_stats_price.csv")

In [ ]:
top6_return_df = top6_df.pct_change()
desc_stats_return = top6_return_df.describe()
desc_stats_return.loc['skewness'] = top6_return_df.skew()
desc_stats_return.loc['kurtosis'] = top6_return_df.kurtosis()

In [ ]:
# desc_stats_return.to_csv("desc_stats_return.csv")

In [ ]:
# for price_df in top6_list:
#     coin_name = price_df.columns[0]
#     title_name = f'{coin_name} Price'
#     plt.figure(figsize=(30, 10));
#     plt.gca().yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
#     plt.ticklabel_format(style='plain', axis='y')
#     plt.plot(price_df);
#     plt.grid(True);
#     plt.title(title_name,fontsize=30);
#     plt.xlabel('Date', fontsize=25);
#     plt.ylabel('Price', fontsize=25);
#     plt.xticks(fontsize=20);
#     plt.yticks(fontsize=20);
#     plt.savefig(f'image/{title_name}.png', dpi=300)
#     plt.clf();
    

In [ ]:
mpl.rcParams['font.size'] = 16  # 기본 폰트 크기 설정
mpl.rcParams['axes.titlesize'] = 20  # 제목 폰트 크기 설정
mpl.rcParams['axes.labelsize'] = 18  # 축 레이블 폰트 크기 설정
mpl.rcParams['xtick.labelsize'] = 14  # x축 눈금 폰트 크기 설정
mpl.rcParams['ytick.labelsize'] = 14  # y축 눈금 폰트 크기 설정
mpl.rcParams['legend.fontsize'] = 16  # 범례 폰트 크기 설정

In [ ]:
for price_ser in top6_list:
    coin_name = price_ser.name
    title_name = f'{coin_name} Return'
    
    plt.figure(figsize=(30, 10));
    
    plt.subplot(1,2,1);
    # sns.set_context("notebook", font_scale=5);
    sns.histplot(price_ser.pct_change(),kde=True, bins=30);
    
    # plt.title(f'{coin_name} Difference', fontsize=30);
    # plt.xlabel(fontsize=25);
    # plt.ylabel(fontsize=25);
    # plt.xticks(fontsize=20);
    # plt.yticks(fontsize=20);
    
    plt.subplot(1,2,2);
    plt.plot(price_ser.pct_change());
    plt.title(f'{coin_name} Return');
    plt.title(title_name);
    plt.xlabel('Date');
    plt.ylabel('Return');
    plt.xticks(rotation=45);
    plt.grid(True);
    plt.tight_layout();
    # plt.show()
    plt.savefig(f'image/{title_name}.png', dpi=300)
    plt.clf();
    

In [ ]:
log_price_df = np.log(price_df)

In [ ]:
# cnt = 1

# for rolling_df in log_price_df.rolling(window=252, step=21):
#     if rolling_df.shape[0] < 252: continue
#     s, e = rolling_df.index[0], rolling_df.index[-1]
#     print(f'Rolling #{cnt}: {s}부터 {e}까지 시작', end=' ')
    
#     corr_df = rolling_df.corr()
#     corr_df = corr_df.reset_index().melt(id_vars='index', value_vars=corr_df.columns)
#     corr_df.columns =['coin1', 'coin2', 'corr']
#     corr_df = corr_df[corr_df['corr']!=1].sort_values(by='corr', ascending=False).drop_duplicates(subset='corr')
#     corr_df = corr_df.reset_index(drop=True)
    
#     adf_df = {
#         "ADF Test Statistic": [],
#         'p-value': [],
#         'Half Life': []
#     }
    
#     for pair in corr_df.values:
#         coin1, coin2, _ = pair
#         coin1_df = rolling_df[coin1]
#         coin2_df = rolling_df[coin2]
#         coin1_log_price = coin1_df.values
#         coin2_log_price = coin2_df.values
        
#         # 선형 회귀 모델 적합
#         X = coin1_log_price
#         Y = coin2_log_price

#         model = sm.OLS(Y, X).fit()
#         beta = model.params
#         ut = Y - (beta * X)
        
#         adf_res = adfuller(ut)
#         model = sm.tsa.ARIMA(ut, order=(1,0,0)).fit()
#         phi_hat = model.arparams[0]
#         half_life = np.log(2) / np.log(1 / abs(phi_hat))
        
#         adf_df["ADF Test Statistic"].append(adf_res[0])
#         adf_df['p-value'].append(adf_res[1])
#         adf_df['Half Life'].append(half_life)
        
#     adf_df = pd.DataFrame(adf_df)
#     corr_df = pd.concat([corr_df, adf_df], axis=1)
#     corr_df = corr_df.loc[corr_df['p-value'] < 0.02, :].reset_index(drop=True)
#     corr_df['corr rank'] = corr_df['corr'].rank(ascending=False)
#     corr_df['p-value rank'] = corr_df['p-value'].rank()
#     corr_df['Half Life rank'] = corr_df['Half Life'].rank()
#     corr_df['total rank'] = corr_df['corr rank'] + corr_df['p-value rank'] + corr_df['Half Life rank']
#     corr_df = corr_df.sort_values(by='total rank').reset_index(drop=True)
    
#     print(f'최종 {corr_df.shape[0]}개 페어 남음')

#     corr_df.to_csv(f'data/corr_{cnt}.csv', index=False)
    
#     del corr_df
    
#     cnt += 1



In [ ]:
top_n = 100

rolling_num = 0

total_return = np.zeros(56) # rolling 평균 성과
total_std = np.zeros(56)

for rolling_df in log_price_df.rolling(window=252, step=21):
    
    rolling_pair_return = np.zeros(top_n) # rolling 할 때 페어별 성과
    # rolling_pair_n_open = np.zeros(top_n) # rolling 할 때 페어별 오픈 횟수
    # rolling_pair_n_close = np.zeros(top_n) # rolling 할 때 페어별 클로즈 횟수
    
    if rolling_df.shape[0] < 252: continue
        
    trading_start = rolling_df.index[-1] + pd.Timedelta(hours=8)
    trading_index = pd.date_range(start=trading_start, periods=126, freq='8H')
    
    last_time_index = log_price_df.index[-1]
    if trading_index[-1] > last_time_index:
        break
    
    
    trading_df = log_price_df.loc[trading_index, :]

    _dir = f'data/corr_{rolling_num+1}.csv'
    top_df = pd.read_csv(_dir)
    pair_names = top_df.loc[:top_n-1:][['coin1', 'coin2']]
    pair_names = pair_names.values

    # rolling_total_open = 0
    
    pair_index = 0
    for c1, c2 in pair_names:
        pair_ret = 0
        
        first_asset_name = c1
        second_asset_name = c2
        in_X = rolling_df[first_asset_name]
        in_Y = rolling_df[second_asset_name]
        out_X = trading_df[first_asset_name]
        out_Y = trading_df[second_asset_name]

        in_model = sm.OLS(in_X, in_Y).fit()
        beta = in_model.params[0]

        in_coint_price = in_X - (beta * in_Y)
        out_coint_price = out_X - (beta * out_Y)

        insample_sd = np.std(in_coint_price)
        index = np.where(
            np.abs(out_coint_price) > (2*insample_sd)
        )[0]

        if len(index) != 0:
            # pair_open += 1
            
            pair_ind = out_coint_price[index[0]] < 0
            
            ind1 = int(pair_ind) - int(not pair_ind)
            ind2 = int(not pair_ind) - int(pair_ind)
            
            W = np.array([
                ind1 * (1 / out_X[index[0]]),
                ind2 * (1 / out_Y[index[0]])
            ])
            
            pair_port = np.dot(W, np.vstack([
                out_X[index[0]:],
                out_Y[index[0]:]
            ]))
            ind_sell = np.where(
                out_coint_price[index[0]:] * out_coint_price[index[0]] <= 0
            )[0]

            if len(ind_sell) == 0:
                pair_ret = pair_port[-1]
            else:
                pair_ret = pair_port[min(ind_sell[0] + 1, len(pair_port) - 1)]
                # pair_n_sell += 1
        else:
            pair_ret = 0

        rolling_pair_return[pair_index] = pair_ret * (1- 0.001) # 수수료 계산
        pair_index += 1
        
    total_return[rolling_num] = np.mean(rolling_pair_return)
    total_std[rolling_num] = np.std(rolling_pair_return)
    rolling_num += 1

In [ ]:
np.mean(total_return) / 42 * 365
np.mean(total_std) / np.sqrt(42 * 365)

In [ ]:
ind_sell = np.where(out_coint_price[index[0]:] * out_coint_price[index[0]] <= 0)[0]
ind_sell

In [ ]:
min(ind_sell[0]+1, len(pair_port)-1)
pair_port[63]

In [ ]:
ind_sell = 


In [ ]:
plt.plot(pair_port)
plt.title(f'Pair Returns (Window {rolling_num})')
plt.axhline(y=0, color='blue', linestyle='--')
plt.grid(True)

In [ ]:
(-1/out_X[15]) * (out_X[15])
